# Importing Libarys 

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# 2. Datasets

1. French Gouverment 2013:
2. French Gouverment 2014:
3. EAA

In [1]:
# Dataset French Gouv 2013
fg_2013 = pd.read_csv("Data/cl_JUIN_2013-complet3.csv", 
                      sep=';', encoding='latin-1')

display(fg_2013.info())

# Dataset French Gouv 2013
fg_2014 = pd.read_csv("Data/mars-2014-complete.csv", 
                      sep=';', encoding='latin-1')

display(fg_2014.info())
display(fg_2014.head())

NameError: name 'pd' is not defined

## 2.1 Column Mapping Dictonary

In [ ]:
column_name_mapping = {
    'Marque': 'Brand',
    'Modèle dossier': 'Model file',
    'Modèle UTAC': 'UTAC model',
    'Désignation commerciale': 'Commercial designation',
    'CNIT': 'CNIT',
    'Type Variante Version (TVV)': 'Type Variant Version (TVV)',
    'Carburant': 'Fuel',
    'Hybride': 'Hybrid',
    'Puissance administrative': 'Administrative power',
    'Puissance maximale (kW)': 'Maximum power (kW)',
    'Boîte de vitesse': 'Transmission',
    'Consommation urbaine (l/100km)': 'Urban consumption (l/100km)',
    'Consommation extra-urbaine (l/100km)': 'Extra-urban consumption (l/100km)',
    'Consommation mixte (l/100km)': 'Combined consumption (l/100km)',
    'CO2 (g/km)': 'CO2 (g/km)',
    'CO type I (g/km)': 'CO type I (g/km)',
    'HC (g/km)': 'HC (g/km)',
    'NOX (g/km)': 'NOX (g/km)',
    'HC+NOX (g/km)': 'HC+NOX (g/km)',
    'Particules (g/km)': 'Particulates (g/km)',
    'masse vide euro min (kg)': 'Empty mass euro min (kg)',
    'masse vide euro max (kg)': 'Empty mass euro max (kg)',
    'Champ V9': 'Field V9',
    'Date de mise à jour': 'Update date',
    'Carrosserie': 'Body',
    'gamme': 'Range'
    }

Column Translation `fg_2013` and preperation

In [ ]:
# Rename the columns using the mapping dictionary
fg_2013.rename(columns=column_name_mapping, inplace=True)

# Display the head of the DataFrame after translation and removal of unnamed columns
display(fg_2014.head())

# Display summary statistics of the DataFrame
display(fg_2014.describe())

# Display information about the DataFrame (data types, non-null counts, etc.)
display(fg_2014.info())

Column Translation `fg_2014` and preperation

In [ ]:
# Translation of column names from French to English

    # Add translations for other columns as needed


# Rename the columns using the translation dictionary
fg_2014.rename(columns=column_translation, inplace=True)

# Remove the unnamed columns (assuming they are empty)
fg_2014 = fg_2014.loc[:, ~fg_2014.columns.str.startswith('Unnamed')]

# Display the head of the DataFrame after translation and removal of unnamed columns
display(fg_2014.head())

# Display summary statistics of the DataFrame
display(fg_2014.describe())

# Display information about the DataFrame (data types, non-null counts, etc.)
display(fg_2014.info())



# 3. Data Exploration